### Домашнее задание «Рекомендации на основе скрытых факторов»
Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)

In [1]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares

In [51]:
user_taggedbookmarks = pd.read_csv('./data/Delicious Bookmarks/user_taggedbookmarks.dat', sep='\t')
bookmarks = pd.read_csv('./data/Delicious Bookmarks/bookmarks.dat', sep='\t', encoding='koi8-r')

In [52]:
bookmarks

,id,md5,title,url,md5Principal,urlPrincipal
0,1,ab4954b633ddaf5b5bba6e9b71aa6b70,IFLA - The official website of the Internation...,http://www.ifla.org/,7f431306c428457bc4e12b15634484f,www.ifla.org
1,2,2221e9cd106d269dd34682666f576fa3,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf,1ef8cfcfe968101fa9b4e301847503d4,archive.ifla.org
2,7,c97c571dadaddbbb493126a0d4d01ba3,EdSelect,http://www.edselect.com/,792fd7eb20143386d0c4eb193c6124d,www.edselect.com
3,8,25bfe8dca0ef263ec9c341b9f16c38b5,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...,6fce4f6391516f0732531d9cfacda5b7,www.collectionscanada.gc.ca
4,9,c97284629e17b8e2861afaacd59918bc,Kidsreads.com,http://www.kidsreads.com/,5854ce8404857a45373eea01a3d98000,www.kidsreads.com
5,10,5a9bdcf2a5ee18c210cdce663b631340,Kidsreads.com -- GREAT BOOKS FOR BOYS,http://www.kidsreads.com/features/great-books-...,5854ce8404857a45373eea01a3d98000,www.kidsreads.com
6,11,d49b0ae16c137b452a148488a400924f,Moodle ::|:: SDUSD Educational Technology,http://moodle.sandi.net/,c40a51ae904e35fbf4e5eb2fc15f95c,moodle.sandi.net
7,14,67e4e670037229c5f36d9450001d3e97,CountryReports.org Б─■ Country Information,http://countryreports.org/,b2afa8146708c6c9957c7e4db29a7814,countryreports.org
8,15,4e72a2a5f48b1a6d2d1c6e8e3e8eb399,"Some of the Best Books, K-12",http://www.kn.att.com/wired/fil/pages/listbest...,fcaf87c40c53c57de95abb9678ca138a,www.kn.att.com
9,16,2c344d706ad1ebdfe0545a26e483b017,Education World б╝: Lesson Plan: Lessons from ...,http://www.educationworld.com/a_lesson/lesson2...,78e74cba021bd118110d7d5347209493,www.educationworld.com


In [37]:
added_bkmrks = user_taggedbookmarks.groupby(by=['userID', 'bookmarkID']).count().reset_index()[['userID', 'bookmarkID']]

In [38]:
added_bkmrks['usr_num'] = added_bkmrks.userID.astype("category").cat.codes
added_bkmrks['bmk_num'] = added_bkmrks.bookmarkID.astype("category").cat.codes

In [39]:
added_bkmrks.head()

,userID,bookmarkID,usr_num,bmk_num
0,8,1,0,0
1,8,2,0,1
2,8,7,0,2
3,8,8,0,3
4,8,9,0,4


In [40]:
users = list(added_bkmrks.userID.unique())
bkmrks = list(added_bkmrks.bookmarkID.unique())
data = [1 for i in range(len(added_bkmrks))]

In [41]:
rows = added_bkmrks.usr_num.astype(int)
cols = added_bkmrks.bmk_num.astype(int)

In [42]:
data_sparse = sparse.csr_matrix((data, (rows, cols)), shape=(len(users), len(bkmrks)))

In [43]:
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|██████████| 15.0/15 [00:01<00:00,  8.61it/s]


In [44]:
user_num = 100

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(user_num, user_items)

In [45]:
recommendations

[(1458, 0.07693196),
 (4, 0.06071826),
 (3, 0.0594838),
 (86, 0.048334964),
 (1550, 0.042942036),
 (1459, 0.027911505),
 (1591, 0.02513282),
 (125, 0.02337068),
 (1561, 0.019855663),
 (335, 0.019712143)]

In [46]:
added_bkmrks[added_bkmrks.usr_num == 100].bmk_num.unique()

array([1310, 1664, 1903, 2831, 2975, 3023, 5672, 6498, 6499, 6500, 6501,
       6502, 6503, 6504, 6505, 6506, 6507, 6508, 6509, 6510, 6511, 6512,
       6513, 6514, 6515, 6516, 6517, 6518, 6519, 6520, 6521, 6522, 6523,
       6524, 6525, 6526, 6527, 6528, 6529, 6530, 6531, 6532, 6533, 6534,
       6535, 6536, 6537, 6538, 6539, 6540, 6541, 6542, 6543, 6544, 6545,
       6546, 6547, 6548, 6549, 6550, 6551, 6552, 6553, 6554, 6555, 6556,
       6557, 6558, 6559, 6560, 6561, 6562, 6563, 6564, 6565, 6566, 6567,
       6568, 6569, 6570])

К сожалению не могу отобразить ссылки, т.к. у меня не получилось загрузить 'bookmarks.dat'.

Что я только не пробовал и где я только не гуглил - пишет:

'utf-8' codec can't decode byte 0xe3 in position 133: unexpected end of data